# PART 3: Connect with `City` Agent for remote SMC

In [ ]:
%%javascript
document.title="Manufacturer1 Agent"

### Imports

In [ ]:
from aries_cloudcontroller import AriesAgentController
import os
#from termcolor import colored
#from pprintpp import pprint
import libs.functions as fnc

### Initialise the Agent Controller

In [ ]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

### Start a Webhook Server

In [ ]:
agent_controller = AriesAgentController(admin_url,api_key)
print(f"Initialising an ACA-PY controller with admin api at {admin_url} and an api key of {api_key}")

await agent_controller.init_webhook_server(webhook_host, webhook_port)
print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

In [ ]:
# Verify if VC "M1-isManufacturer-VC" already exists in wallet
#vc = "M1-isManufacturer-VC"
#fnc.is_vc_in_wallet(agent_controller, vc)

## Register Agent Event Listeners

You can see some examples within the webhook_listeners recipe. Copy any relevant cells across and customise as needed.

In [ ]:
listeners = []
     
connection_listener = {"handler": fnc.connections_handler, "topic": "connections"}
listeners.append(connection_listener)

prover_listener = {"topic": "present_proof", "handler": fnc.prover_proof_handler}
listeners.append(prover_listener)

agent_controller.register_listeners(listeners)

## Accept Invitation

Copy an invitation object from another agent playing the role inviter (see the inviter_template recipe)

In [ ]:
invitation = input("Please enter invitation from external agent: ")

In [ ]:
auto_accept="true"
alias=None

invite_response = await agent_controller.connections.receive_invitation(invitation, alias, auto_accept)
connection_id = invite_response["connection_id"]

## Optional: Send Proposal

Propose a presentation to a verifier

In [ ]:
# TODO: Example proposal object below

# proposal_object = {
#   "auto_present": true,
#   "comment": "string",
#   "connection_id": "3fa85f64-5717-4562-b3fc-2c963f66afa6",
#   "presentation_proposal": {
#     "@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/presentation-preview",
#     "attributes": [
#       {
#         "cred_def_id": "WgWxqztrNooG92RXvxSTWv:3:CL:20:tag",
#         "mime-type": "image/jpeg",
#         "name": "favourite_drink",
#         "referent": "0",
#         "value": "martini"
#       }
#     ],
#     "predicates": [
#       {
#         "cred_def_id": "WgWxqztrNooG92RXvxSTWv:3:CL:20:tag",
#         "name": "high_score",
#         "predicate": ">=",
#         "threshold": 0
#       }
#     ]
#   },
#   "trace": false
# }

# proposal_response = await agent_controller.proofs.send_proposal(proposal_object)

## Fetch Presentation Records

Before you can present a presentation, you must identify the presentation record which you wish to respond to with a presentation. This could also be done through the present_proof listeners which have access to a presentation record in the payload.

In [ ]:
# Step 2

# Optional Query parameters
verifier_connection_id = connection_id
thread_id = None
state = "request_received"
role = "prover"

proof_records_response = await agent_controller.proofs.get_records(verifier_connection_id, thread_id, state, role)
pprint(proof_records_response)

In [ ]:
# Fetch records from response that were sent via connection_id
presentation_records = [r for r in proof_records_response["results"] if (r["connection_id"] == connection_id) is True] 

# Process only the first record:
presentation_record = presentation_records[0]
presentation_exchange_id = presentation_record["presentation_exchange_id"]

## Search For Available Credentials to Construct Presentation From

The presentation record can be used to query your agents wallet and return all credentials that could be used to construct valid presentation

In [ ]:
def get_proof_request_requirements(presentation_record):
    """
    Returns dictionary with {<required-attribute>: <restrictions-of-attribute>} from presentation record
    """
    # Setup
    restrictions = {}
    presentation_request = presentation_record["presentation_request"]
    
    # Get required attributes and requirements for the individual attributes
    for attr_key, attr_val in presentation_request["requested_attributes"].items():
        restrictions[attr_val["name"]] = {}
        restrictions[attr_val["name"]]["requirements"] = attr_val["restrictions"][0] 
        restrictions[attr_val["name"]]["request_attr_name"] = attr_key
        
    return restrictions


def get_suitable_credentials(credentials_all, requirements):
    """
    Finds credentials in credentials_all that satisfy the requirements provided by the relying party.
    Returns dictionary with: {<attribute-name>: <suitable-credential>}, where the suitable-credential satisfies all requirements 
    """
    # Setup
    relevant_credentials = {}
    revealed = {}
    credentials = credentials_all["results"]
    
    # Iterate through attribute name and attribute requirements of relying party
    for name, conditions in requirements.items():
        
        req = conditions["requirements"]
        req_name = conditions["request_attr_name"]
        
        # Break if the required attribute name is not in any credential, or if all requirements (e.g., schema_id) are not within one credential
        if (any(name in cred["attrs"] for cred in credentials) is False) or (any(r in cred.keys() for r in req for cred in credentials) is False):
            continue

        # Iterate through credentials
        for cred in credentials:

            # Verify if requirement value (r_val) and credential value (cred[r_key]) match for required attribute (r_key)
            for r_key, r_val in req.items():
                try:
                    # Append cred to relevant_credentials if all requirements match
                    if (cred[r_key] == r_val) is True:
                        relevant_credentials[name] = cred
                        print(f"Attribute request for '{name}' can be satisfied by Credential with Referent -- {cred['referent']}")
                        revealed[req_name] = {"cred_id": cred["referent"], "revealed": True}
                except:
                    pass

    return relevant_credentials, revealed

In [ ]:
# Get requirements of proof request
proof_attribute_requirements = get_proof_request_requirements(presentation_record)

# Verify all credentials of M1 and verify if they satisfy the requirements defined in proof_attribute_requirements
credentials = await agent_controller.credentials.get_all()
suitable_credentials, revealed = get_suitable_credentials(credentials, proof_attribute_requirements)
#pprint(credentials)
pprint(revealed)

In [ ]:
# select credentials to provide for the proof
#credentials = await agent_controller.proofs.get_presentation_credentials(presentation_exchange_id)
#print("Credentials stored that could be used to satisfy the request. In some situations you applications may have a choice which credential to reveal\n")

#attribute_by_reft = {}
#revealed = {}
#self_attested = {}
#predicates = {}

In [ ]:
predicates = {}
self_attested = {}


In [ ]:
# Note we are working on a friendlier api to abstract this away

print("\nGenerate the proof")
presentation = {
    "requested_predicates": predicates,
    "requested_attributes": revealed,
    "self_attested_attributes": self_attested,
}
print(presentation)

## Send Presentation

A presentation is sent in represent to a presentation record that has previously been created.

In [ ]:
presentation_response = await agent_controller.proofs.send_presentation(presentation_exchange_id, presentation)

## Your Own Business Logic

Now you should have an established, active connection you can write any custom logic you want to engage with protocols with the connection

In [ ]:
# Imports
import torch as th
import syft as sy

from sympc.session import Session
from sympc.session import SessionManager
from sympc.tensor import MPCTensor

In [ ]:
# Copy and paste client_id and ping the city!

In [ ]:
# Step 3: Create duet server
duet_m1 = sy.duet()

## 2 - Secure Multi-Party Computation

In [ ]:
from IPython.core.debugger import set_trace

In [ ]:
# Step 8: test there is no data in the server atm
duet_m1.store.pandas

In [ ]:
age_data = th.tensor([50,23,72,83])
age_data = age_data.tag("test_data_age")
age_data = age_data.describe("description for age_data")
age_data_pointer = age_data.send(duet_m1, pointable=True)

In [ ]:
duet_m1.store

In [ ]:
duet_m1.store.pandas

In [ ]:
duet_m1.id

In [ ]:
duet_m1.name

In [ ]:
duet_m1.target_id

In [ ]:
duet_m1.network_id

In [ ]:
#duet_city.store.pandas

In [ ]:
# Step 9: Publish secret data

x = torch.tensor([50,60,77]) # define torch
x.send(duet_m1, pointable=True, tags=["TEST_#1_from_m1_to_ds"], description="Dummy data") # send torch to duet_m1

x = torch.tensor([[1,2], [3,4]])
x.send(duet_m1, pointable=True, tags=["TEST_#2_from_m1_to_ds"], description="Test to multiply matrices")

In [ ]:
# Step 10: verify that data was uploaded to the store
duet_m1.store.pandas

In [ ]:
# Step 11: Authorize the requests ds can make
duet_m1.requests.add_handler(action="accept")

In [ ]:
# Step 11: Authorize the requests ds can make
duet_city.requests.add_handler(action="accept")

## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [ ]:
await agent_controller.terminate()